In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel

In [2]:
BATCH_SIZE = 64

In [3]:
path = "/content/drive/MyDrive/Thesis/Datasets/Turbofan_Dataset/final_datasets_normalized/"

In [4]:
# Load data and drop irrelevant columns

df_train = pd.read_csv(path + "TRAINING_SET_FULL.csv")
df_test = pd.read_csv(path + "TEST_SET_FULL.csv")

drop_cols = ["id","cycle","setting3","s1","s5","s10","s16","s18","s19","RUL"]

corr_cols = ["s11","s4","s15","s17","s2","s3","s8","s13","s9","s14","s12","s7","s20"]

train_set = df_train.drop(drop_cols, axis=1)
test_set = df_test.drop(drop_cols, axis=1)

In [5]:
# # Adding lagged variables
# for col in corr_cols:
#   train_set[col+"_lag_1"] = train_set[col].shift(1)
#   train_set[col+"_lag_2"] = train_set[col].shift(2)

#   test_set[col+"_lag_1"] = test_set[col].shift(1)
#   test_set[col+"_lag_2"] = test_set[col].shift(2)

# # Removing NaN values

# train_set.dropna(inplace=True)
# test_set.dropna(inplace=True)

In [6]:
# Move cycle_norm column first for convenience

column_to_move = train_set.pop("cycle_norm")
train_set.insert(0, "cycle_norm", column_to_move)

column_to_move = test_set.pop("cycle_norm")
test_set.insert(0, "cycle_norm", column_to_move)

In [7]:
# Convert pandas dataframes to numpy arrays

X_train = train_set.iloc[:, 0:-1].to_numpy()
y_train = train_set.iloc[:, -1].to_numpy()
X_test = test_set.iloc[:, 0:-1].to_numpy()
y_test = test_set.iloc[:, -1].to_numpy()

In [8]:
# poly = PolynomialFeatures(2)
# X_train = poly.fit_transform(X_train)
# X_test = poly.fit_transform(X_test)

In [9]:
# Defining custom dataset class for convenience

class CustomDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [10]:
# Initialize custom datasets

train_data = CustomDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data = CustomDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

In [11]:
# Initialize dataloaders

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
# Check how many inputs and outputs the model should have

print(f"Our model should have {train_loader.dataset.X_data.shape[-1]} inputs and 1 output")

Our model should have 18 inputs and 1 output


In [13]:
inputs = train_loader.dataset.X_data.shape[-1]
inputs

18

In [14]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 18.
        self.layer_1 = nn.Linear(inputs, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [16]:
EPOCHS = 20
LEARNING_RATE = 0.001


In [17]:
model = BinaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

BinaryClassification(
  (layer_1): Linear(in_features=18, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [18]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [19]:
# Model training

model.train() #tells pytorch that we are in training mode

y_pred_train_list = []

for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)

        y_train_pred = torch.sigmoid(y_pred)
        y_pred_tag = torch.round(y_train_pred)
        y_pred_train_list.append(y_pred_tag.cpu().detach().numpy())
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

y_pred_train_list = [a.squeeze().tolist() for a in y_pred_train_list]
y_pred_train_list = [item for sublist in y_pred_train_list for item in sublist]

Epoch 001: | Loss: 0.34457 | Acc: 86.777
Epoch 002: | Loss: 0.13265 | Acc: 95.972
Epoch 003: | Loss: 0.11013 | Acc: 96.300
Epoch 004: | Loss: 0.10101 | Acc: 96.533
Epoch 005: | Loss: 0.09781 | Acc: 96.663
Epoch 006: | Loss: 0.09651 | Acc: 96.749
Epoch 007: | Loss: 0.09380 | Acc: 96.916
Epoch 008: | Loss: 0.09016 | Acc: 96.944
Epoch 009: | Loss: 0.08973 | Acc: 97.006
Epoch 010: | Loss: 0.08726 | Acc: 97.102
Epoch 011: | Loss: 0.08717 | Acc: 97.121
Epoch 012: | Loss: 0.08626 | Acc: 97.180
Epoch 013: | Loss: 0.08515 | Acc: 97.133
Epoch 014: | Loss: 0.08555 | Acc: 97.111
Epoch 015: | Loss: 0.08408 | Acc: 97.198
Epoch 016: | Loss: 0.08125 | Acc: 97.269
Epoch 017: | Loss: 0.07958 | Acc: 97.331
Epoch 018: | Loss: 0.07916 | Acc: 97.455
Epoch 019: | Loss: 0.07870 | Acc: 97.365
Epoch 020: | Loss: 0.07697 | Acc: 97.486


In [20]:
# print(classification_report(y_train, y_pred_train_list))

In [21]:
len(y_pred_train_list)

412620

In [22]:
len(y_train)

20631

In [23]:
# Model testing

model.eval()

y_pred_test_list = []


with torch.no_grad():

  test_loss = 0
  test_accuracy = 0

  for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        y_test_pred = model(X_batch)
        
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_test_list.append(y_pred_tag.cpu().numpy())
        
        loss = criterion(y_pred_tag, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred_tag, y_batch.unsqueeze(1))
        
        test_loss += loss.item()
        test_accuracy += acc.item()

y_pred_test_list = [a.squeeze().tolist() for a in y_pred_test_list]
y_pred_test_list = [item for sublist in y_pred_test_list for item in sublist]


print(f'Test set evaluation : | Loss: {test_loss/len(test_loader):.5f} | Acc: {test_accuracy/len(test_loader):.3f}')

Test set evaluation : | Loss: 0.69628 | Acc: 97.629


In [24]:
print(classification_report(y_test, y_pred_test_list))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     12764
           1       0.53      0.72      0.61       332

    accuracy                           0.98     13096
   macro avg       0.76      0.85      0.80     13096
weighted avg       0.98      0.98      0.98     13096



In [25]:
# polyfit no batchnorm no dropout
print(classification_report(y_test, y_pred_test_list))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     12764
           1       0.53      0.72      0.61       332

    accuracy                           0.98     13096
   macro avg       0.76      0.85      0.80     13096
weighted avg       0.98      0.98      0.98     13096



In [26]:
# no batchnorm and dropout
print(classification_report(y_test, y_pred_test_list))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     12764
           1       0.53      0.72      0.61       332

    accuracy                           0.98     13096
   macro avg       0.76      0.85      0.80     13096
weighted avg       0.98      0.98      0.98     13096



In [27]:
# batchnorm and dropout
# print(classification_report(y_test, y_pred_test_list))